In [ ]:
# ✅ STEP 1: Install requirements
!pip install segmentation-models-pytorch --quiet
!pip install albumentations --quiet

import torch
import numpy as np
import matplotlib.pyplot as plt
import cv2
from torchvision import transforms
from PIL import Image
import os
from google.colab import files



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 888.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 952.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 12.4 MB/s eta 0:00:00


In [ ]:
import os, zipfile


In [ ]:
import segmentation_models_pytorch as smp

In [ ]:
# 1. Upload ZIP of images
from google.colab import files
uploaded = files.upload()  # upload your RBG_JPEG.zip

Saving RBG_JPEG.zip to RBG_JPEG (2).zip


In [ ]:
# 2. Extract ZIP

zip_path = "/content/RBG_JPEG (2).zip"  # Corrected filename
extract_dir = "/content/facade_images"
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

In [ ]:
# List all files directly in the folder
for root, dirs, files in os.walk(extract_dir):
    for file in files:
        print(os.path.join(root, file))

/content/facade_images/RBG_JPEG/100050961224-rgb_2024-03-27-08-44-54.jpeg
/content/facade_images/RBG_JPEG/100050960731-rgb_2024-03-27-08-50-04.jpeg
/content/facade_images/RBG_JPEG/200003015323-rgb_2024-03-27-08-58-33.jpeg
/content/facade_images/RBG_JPEG/100050960725-rgb_2024-03-27-08-50-02.jpeg
/content/facade_images/RBG_JPEG/100050961195-rgb_2024-03-27-08-44-42.jpeg
/content/facade_images/RBG_JPEG/100050961213-rgb_2024-03-27-08-44-42.jpeg
/content/facade_images/RBG_JPEG/100050951099-rgb_2024-03-27-08-54-25.jpeg
/content/facade_images/RBG_JPEG/100050960745-rgb_2024-03-27-08-50-09.jpeg
/content/facade_images/RBG_JPEG/100050950913-rgb_2024-03-27-08-53-37.jpeg
/content/facade_images/RBG_JPEG/100050961198-rgb_2024-03-27-08-44-44.jpeg
/content/facade_images/RBG_JPEG/200000564694-rgb_2024-03-25-17-17-45.jpeg
/content/facade_images/RBG_JPEG/10093467883-rgb_2024-03-27-08-47-40.jpeg
/content/facade_images/RBG_JPEG/100050950969-rgb_2024-03-27-08-53-26.jpeg
/content/facade_images/RBG_JPEG/2000030

In [ ]:


# 3. Prepare model
model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=12
)
model.eval()  # No weights loaded (placeholder only)

# 4. Define color palette and labels
facade_palette = np.array([
    [0, 0, 0],        # 0 Background
    [128, 0, 0],      # 1 Wall
    [0, 128, 0],      # 2 Window
    [128, 128, 0],    # 3 Door
    [0, 0, 128],      # 4 Cornice
    [128, 0, 128],    # 5 Sill
    [0, 128, 128],    # 6 Balcony
    [128, 128, 128],  # 7 Blind
    [64, 0, 0],       # 8 Pillar
    [192, 0, 0],      # 9 Shop
    [64, 128, 0],     # 10 Molding
    [192, 128, 0]     # 11 Deco
])
facade_labels = [
    "Background", "Wall", "Window", "Door", "Cornice", "Sill",
    "Balcony", "Blind", "Pillar", "Shop", "Molding", "Deco"
]

# 5. Get all image files (recursive and case-insensitive)
from glob import glob
image_files = sorted(glob(f"{extract_dir}/**/*.[jJ][pP][gG]", recursive=True)) + \
              sorted(glob(f"{extract_dir}/**/*.[jJ][pP][eE][gG]", recursive=True)) + \
              sorted(glob(f"{extract_dir}/**/*.[pP][nN][gG]", recursive=True))
print(f"✅ Found {len(image_files)} image(s) in {extract_dir}")

# 6. Output folder
os.makedirs("facade_outputs", exist_ok=True)

# 7. Inference + overlay generation
from PIL import Image, ImageDraw, ImageFont
for path in image_files:
    name = os.path.splitext(os.path.basename(path))[0]
    image = Image.open(path).convert("RGB").resize((512, 512))
    img_array = np.array(image) / 255.0
    input_tensor = torch.tensor(img_array).permute(2, 0, 1).unsqueeze(0).float()

    with torch.no_grad():
        pred = model(input_tensor)[0].argmax(0).cpu().numpy()

    # Save colorized mask
    color_mask = facade_palette[pred].astype(np.uint8)
    color_image = Image.fromarray(color_mask)
    color_image.save(f"facade_outputs/{name}_mask.png")

    # Overlay class names
    overlay = image.copy()
    draw = ImageDraw.Draw(overlay)
    try:
        font = ImageFont.truetype("arial.ttf", 14)
    except:
        font = ImageFont.load_default()

    unique_classes = np.unique(pred)
    for cls_id in unique_classes:
        mask = (pred == cls_id).astype(np.uint8)
        y, x = np.where(mask)
        if len(x) == 0:
            continue
        cx, cy = int(np.mean(x)), int(np.mean(y))
        label = facade_labels[cls_id]
        draw.text((cx + 5, cy + 5), label, fill="white", font=font)

    overlay.save(f"facade_outputs/{name}_overlay.png")
    print(f"✅ Saved {name}_mask and overlay.")


✅ Found 30 image(s) in /content/facade_images
✅ Saved 100050914245-rgb_2024-03-27-08-55-03_mask and overlay.
✅ Saved 100050950866-rgb_2024-03-27-08-54-06_mask and overlay.
✅ Saved 100050950913-rgb_2024-03-27-08-53-37_mask and overlay.
✅ Saved 100050950942-rgb_2024-03-27-08-51-19_mask and overlay.
✅ Saved 100050950969-rgb_2024-03-27-08-53-26_mask and overlay.
✅ Saved 100050951099-rgb_2024-03-27-08-54-25_mask and overlay.
✅ Saved 100050951119-rgb_2024-03-27-08-54-34_mask and overlay.
✅ Saved 100050951128-rgb_2024-03-27-08-54-28_mask and overlay.
✅ Saved 100050960725-rgb_2024-03-27-08-50-02_mask and overlay.
✅ Saved 100050960731-rgb_2024-03-27-08-50-04_mask and overlay.
✅ Saved 100050960733-rgb_2024-03-27-08-50-05_mask and overlay.
✅ Saved 100050960745-rgb_2024-03-27-08-50-09_mask and overlay.
✅ Saved 100050960776-rgb_2024-03-27-08-50-26_mask and overlay.
✅ Saved 100050961195-rgb_2024-03-27-08-44-42_mask and overlay.
✅ Saved 100050961196-rgb_2024-03-27-08-44-44_mask and overlay.
✅ Saved 1

In [ ]:
# ✅ Install dependencies
!pip install -q transformers timm

In [ ]:
from google.colab import files
from glob import glob

In [ ]:
from transformers import SegformerFeatureExtractor, SegformerForSemanticSegmentation

In [ ]:
zip_path = "/content/RBG_JPEG (2).zip"  # Corrected filename
extract_dir = "/content/facade_images"
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# ✅ Step 3: Load pretrained CMP facade model
feature_extractor = SegformerFeatureExtractor.from_pretrained("Xpitfire/segformer-finetuned-segments-cmp-facade")
model = SegformerForSemanticSegmentation.from_pretrained("Xpitfire/segformer-finetuned-segments-cmp-facade").eval()

# 5. Get all image files (recursive and case-insensitive)
from glob import glob
image_files = sorted(glob(f"{extract_dir}/**/*.[jJ][pP][gG]", recursive=True)) + \
              sorted(glob(f"{extract_dir}/**/*.[jJ][pP][eE][gG]", recursive=True)) + \
              sorted(glob(f"{extract_dir}/**/*.[pP][nN][gG]", recursive=True))
print(f"✅ Found {len(image_files)} image(s) in {extract_dir}")

# ✅ Step 5: Output folder
os.makedirs("facade_outputs", exist_ok=True)

# ✅ Step 6: Define color palette and label map
labels = model.config.id2label
# Ensure palette has enough colors for all classes (0 to 12)
palette = [
    [128, 0, 0], [0, 128, 0], [128, 128, 0], [0, 0, 128],
    [128, 0, 128], [0, 128, 128], [128, 128, 128], [64, 0, 0],
    [192, 0, 0], [64, 128, 0], [192, 128, 0], [255, 255, 255], [0, 255, 255] # Added a new color for class 12
]

# ✅ Step 7: Segment and overlay labels
import cv2 # Import cv2 for resizing
for path in image_files:
    name = os.path.splitext(os.path.basename(path))[0]
    image = Image.open(path).convert("RGB").resize((512, 512))
    inputs = feature_extractor(images=image, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)
    seg = outputs.logits.argmax(dim=1)[0].cpu().numpy()

    # Resize the segmentation map to match the original image size
    seg_resized = cv2.resize(seg.astype(np.uint8), (image.width, image.height), interpolation=cv2.INTER_NEAREST)


    # Save class mask
    color_mask = np.zeros((image.height, image.width, 3), dtype=np.uint8)
    for cls_id in np.unique(seg_resized): # Use the resized segmentation map
        # Ensure cls_id is within the bounds of the palette
        if cls_id < len(palette):
             color_mask[seg_resized == cls_id] = palette[cls_id]
        else:
             print(f"Warning: Class ID {cls_id} found but no corresponding color in palette. Using black.")
             color_mask[seg_resized == cls_id] = [0, 0, 0] # Use black for unknown classes
    Image.fromarray(color_mask).save(f"facade_outputs/{name}_mask.png")

    # Overlay
    overlay = image.copy()
    draw = ImageDraw.Draw(overlay)
    font = ImageFont.load_default()
    for cls_id in np.unique(seg_resized): # Use the resized segmentation map
        mask = seg_resized == cls_id
        y, x = np.where(mask)
        if not len(x): continue
        cx, cy = int(x.mean()), int(y.mean())
        # Ensure cls_id is within the bounds of the labels dictionary
        if str(cls_id) in labels:
            label = labels[str(cls_id)]
            draw.text((cx, cy), label, fill="white", font=font)
        else:
            print(f"Warning: Class ID {cls_id} found but no corresponding label.")


    overlay.save(f"facade_outputs/{name}_overlay.png")
    print(f"Saved: {name}_mask.png and overlay.png")

print("\n✅ Done. Masks and overlays saved to 'facade_outputs/' folder.")

/usr/local/lib/python3.11/dist-packages/transformers/models/segformer/feature_extraction_segformer.py:30: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/utils/deprecation.py:172: UserWarning: The following named arguments are not valid for `SegformerFeatureExtractor.__init__` and were ignored: 'feature_extractor_type'
  return func(*args, **kwargs)


✅ Found 30 image(s) in /content/facade_images
Saved: 100050914245-rgb_2024-03-27-08-55-03_mask.png and overlay.png
Saved: 100050950866-rgb_2024-03-27-08-54-06_mask.png and overlay.png
Saved: 100050950913-rgb_2024-03-27-08-53-37_mask.png and overlay.png
Saved: 100050950942-rgb_2024-03-27-08-51-19_mask.png and overlay.png
Saved: 100050950969-rgb_2024-03-27-08-53-26_mask.png and overlay.png
Saved: 100050951099-rgb_2024-03-27-08-54-25_mask.png and overlay.png
Saved: 100050951119-rgb_2024-03-27-08-54-34_mask.png and overlay.png
Saved: 100050951128-rgb_2024-03-27-08-54-28_mask.png and overlay.png
Saved: 100050960725-rgb_2024-03-27-08-50-02_mask.png and overlay.png
Saved: 100050960731-rgb_2024-03-27-08-50-04_mask.png and overlay.png
Saved: 100050960733-rgb_2024-03-27-08-50-05_mask.png and overlay.png
Saved: 100050960745-rgb_2024-03-27-08-50-09_mask.png and overlay.png
Saved: 100050960776-rgb_2024-03-27-08-50-26_mask.png and overlay.png
Saved: 100050961195-rgb_2024-03-27-08-44-42_mask.png and 

In [ ]:
# List all files directly in the folder
for root, dirs, files in os.walk(extract_dir):
    for file in files:
        print(os.path.join(root, file))

/content/facade_images/RBG_JPEG/100050961224-rgb_2024-03-27-08-44-54.jpeg
/content/facade_images/RBG_JPEG/100050960731-rgb_2024-03-27-08-50-04.jpeg
/content/facade_images/RBG_JPEG/200003015323-rgb_2024-03-27-08-58-33.jpeg
/content/facade_images/RBG_JPEG/100050960725-rgb_2024-03-27-08-50-02.jpeg
/content/facade_images/RBG_JPEG/100050961195-rgb_2024-03-27-08-44-42.jpeg
/content/facade_images/RBG_JPEG/100050961213-rgb_2024-03-27-08-44-42.jpeg
/content/facade_images/RBG_JPEG/100050951099-rgb_2024-03-27-08-54-25.jpeg
/content/facade_images/RBG_JPEG/100050960745-rgb_2024-03-27-08-50-09.jpeg
/content/facade_images/RBG_JPEG/100050950913-rgb_2024-03-27-08-53-37.jpeg
/content/facade_images/RBG_JPEG/100050961198-rgb_2024-03-27-08-44-44.jpeg
/content/facade_images/RBG_JPEG/200000564694-rgb_2024-03-25-17-17-45.jpeg
/content/facade_images/RBG_JPEG/10093467883-rgb_2024-03-27-08-47-40.jpeg
/content/facade_images/RBG_JPEG/100050950969-rgb_2024-03-27-08-53-26.jpeg
/content/facade_images/RBG_JPEG/2000030

In [ ]:
import os
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from glob import glob

# Corrected paths
mask_dir = "facade_outputs"
output_dir = "facade_outputs/legended"
os.makedirs(output_dir, exist_ok=True)

# CMP Facade palette and labels
cmp_palette = np.array([
    [0, 0, 0],        # 0 Background
    [128, 0, 0],      # 1 Wall
    [0, 128, 0],      # 2 Window
    [128, 128, 0],    # 3 Door
    [0, 0, 128],      # 4 Cornice
    [128, 0, 128],    # 5 Sill
    [0, 128, 128],    # 6 Balcony
    [128, 128, 128],  # 7 Blind
    [64, 0, 0],       # 8 Pillar
    [192, 0, 0],      # 9 Shop
    [64, 128, 0],     # 10 Molding
    [192, 128, 0]     # 11 Deco
], dtype=np.uint8)

cmp_labels = [
    "Background", "Wall", "Window", "Door", "Cornice", "Sill",
    "Balcony", "Blind", "Pillar", "Shop", "Molding", "Deco"
]

# Find mask images
mask_paths = glob(os.path.join(mask_dir, "*_mask.png"))

print(f"Found {len(mask_paths)} mask(s)")

for mask_path in mask_paths:
    name = os.path.splitext(os.path.basename(mask_path))[0].replace("_mask", "")
    mask = Image.open(mask_path).convert("RGB")
    mask_np = np.array(mask)

    # Reverse-map color to class index
    pred = np.zeros((mask_np.shape[0], mask_np.shape[1]), dtype=np.uint8)
    for idx, color in enumerate(cmp_palette):
        matches = np.all(mask_np == color, axis=-1)
        pred[matches] = idx

    # Draw labels on overlay
    overlay = mask.copy()
    draw = ImageDraw.Draw(overlay)
    try:
        font = ImageFont.truetype("arial.ttf", 14)
    except:
        font = ImageFont.load_default()

    for cls_id in np.unique(pred):
        if cls_id >= len(cmp_labels):
            continue
        mask_bin = (pred == cls_id)
        if np.sum(mask_bin) < 100:
            continue
        y_coords, x_coords = np.where(mask_bin)
        cx, cy = int(np.mean(x_coords)), int(np.mean(y_coords))
        draw.text((cx + 5, cy + 5), cmp_labels[cls_id], fill="white", font=font)

    overlay.save(f"{output_dir}/{name}_labeled_overlay.png")
    print(f"✅ Saved: {name}_labeled_overlay.png")


Found 30 mask(s)
✅ Saved: 100050960776-rgb_2024-03-27-08-50-26_labeled_overlay.png
✅ Saved: 100050961224-rgb_2024-03-27-08-44-54_labeled_overlay.png
✅ Saved: 100050950942-rgb_2024-03-27-08-51-19_labeled_overlay.png
✅ Saved: 100050914245-rgb_2024-03-27-08-55-03_labeled_overlay.png
✅ Saved: 10093467883-rgb_2024-03-27-08-47-40_labeled_overlay.png
✅ Saved: 100050961213-rgb_2024-03-27-08-44-42_labeled_overlay.png
✅ Saved: 10095307902-rgb_2024-03-25-17-15-41_labeled_overlay.png
✅ Saved: 100050960733-rgb_2024-03-27-08-50-05_labeled_overlay.png
✅ Saved: 100050961198-rgb_2024-03-27-08-44-44_labeled_overlay.png
✅ Saved: 10025504345-rgb_2024-03-12-11-56-57_labeled_overlay.png
✅ Saved: 100050961223-rgb_2024-03-27-08-44-45_labeled_overlay.png
✅ Saved: 100050960731-rgb_2024-03-27-08-50-04_labeled_overlay.png
✅ Saved: 100050950913-rgb_2024-03-27-08-53-37_labeled_overlay.png
✅ Saved: 100050961195-rgb_2024-03-27-08-44-42_labeled_overlay.png
✅ Saved: 100050951128-rgb_2024-03-27-08-54-28_labeled_overlay.

In [ ]:
folder_path = "facade_outputs/overlays_labeled"

# List all files in the folder
files = os.listdir(folder_path)
print(f"Found {len(files)} file(s) in {folder_path}:")
for f in files:
    print(f)

Found 0 file(s) in facade_outputs/overlays_labeled:


In [ ]:
# Set your directories
mask_dir = "facade_outputs"
output_dir = "facade_outputs/legended_2"
os.makedirs(output_dir, exist_ok=True)

# Get PNG mask files
mask_paths = sorted([f for f in os.listdir(mask_dir) if f.endswith(".png")])
print(f"Found {len(mask_paths)} mask(s)")

for filename in mask_paths:
    mask_path = os.path.join(mask_dir, filename)
    mask_img = Image.open(mask_path).convert("RGB")
    mask_np = np.array(mask_img)

    # Identify present classes in the mask by matching palette colors
    present_classes = []
    for i, color in enumerate(cmp_palette):
        if np.any(np.all(mask_np == color, axis=-1)):
            present_classes.append(i)

    # Plot mask + legend
    plt.figure(figsize=(6, 6))
    plt.imshow(mask_np)
    plt.axis("off")

    legend_elements = [Patch(facecolor=tuple(cmp_palette[i] / 255.0),
                             label=cmp_labels[i]) for i in present_classes]
    plt.legend(handles=legend_elements, bbox_to_anchor=(1.05, 1), loc="upper left", borderaxespad=0.)

    # Save output
    out_path = os.path.join(output_dir, filename.replace(".png", "_legend.png"))
    plt.tight_layout()
    plt.savefig(out_path, bbox_inches="tight")
    plt.close()
    print(f"Saved: {out_path}")

Found 60 mask(s)
Saved: facade_outputs/legended_2/100050914245-rgb_2024-03-27-08-55-03_mask_legend.png
Saved: facade_outputs/legended_2/100050914245-rgb_2024-03-27-08-55-03_overlay_legend.png
Saved: facade_outputs/legended_2/100050950866-rgb_2024-03-27-08-54-06_mask_legend.png
Saved: facade_outputs/legended_2/100050950866-rgb_2024-03-27-08-54-06_overlay_legend.png
Saved: facade_outputs/legended_2/100050950913-rgb_2024-03-27-08-53-37_mask_legend.png
Saved: facade_outputs/legended_2/100050950913-rgb_2024-03-27-08-53-37_overlay_legend.png
Saved: facade_outputs/legended_2/100050950942-rgb_2024-03-27-08-51-19_mask_legend.png
Saved: facade_outputs/legended_2/100050950942-rgb_2024-03-27-08-51-19_overlay_legend.png
Saved: facade_outputs/legended_2/100050950969-rgb_2024-03-27-08-53-26_mask_legend.png
Saved: facade_outputs/legended_2/100050950969-rgb_2024-03-27-08-53-26_overlay_legend.png
Saved: facade_outputs/legended_2/100050951099-rgb_2024-03-27-08-54-25_mask_legend.png
Saved: facade_outputs/

In [ ]:
# prompt: download a zipfile of folder legended_2

import os # Import os for path joining

# Zip the folder
zip_output_path = "/content/facade_outputs/legended_2.zip" # Corrected output path
!zip -r {zip_output_path} facade_outputs/legended_2/

# Download the zip file
from google.colab import files # Import files if not already imported
files.download(zip_output_path)

  adding: facade_outputs/legended_2/ (stored 0%)
  adding: facade_outputs/legended_2/100050950866-rgb_2024-03-27-08-54-06_overlay_legend.png (deflated 0%)
  adding: facade_outputs/legended_2/10091128670-rgb_2024-03-27-08-49-11_overlay_legend.png (deflated 0%)
  adding: facade_outputs/legended_2/100050961213-rgb_2024-03-27-08-44-42_mask_legend.png (deflated 8%)
  adding: facade_outputs/legended_2/100050961196-rgb_2024-03-27-08-44-44_overlay_legend.png (deflated 0%)
  adding: facade_outputs/legended_2/10025504345-rgb_2024-03-12-11-56-57_mask_legend.png (deflated 9%)
  adding: facade_outputs/legended_2/100050914245-rgb_2024-03-27-08-55-03_mask_legend.png (deflated 8%)
  adding: facade_outputs/legended_2/100050961223-rgb_2024-03-27-08-44-45_mask_legend.png (deflated 8%)
  adding: facade_outputs/legended_2/200000564694-rgb_2024-03-25-17-17-45_mask_legend.png (deflated 10%)
  adding: facade_outputs/legended_2/10091128648-rgb_2024-03-27-08-47-37_overlay_legend.png (deflated 0%)
  adding: faca

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>